# TM10007 Assignment template

In [1]:
# Run this to use from colab environment
%pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

Note: you may need to restart the kernel to use updated packages.


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [2]:
# Data loading functions. Uncomment the one you want to use
from worcliver.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 186
The number of columns: 494


# Data clean before split
Removal of dubble entries (Features and samples)